In [8]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

from sklearn.decomposition import TruncatedSVD

nltk.download('stopwords')
nltk.download('punkt')

import gensim
import gensim.downloader as api
from gensim.test.utils import get_tmpfile
from gensim.models import Word2Vec

morph_analyzer = MorphAnalyzer()
model_ruscorpora = api.load('word2vec-ruscorpora-300')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Xenus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Xenus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
stop_words = stopwords.words('russian')

In [4]:
def lemmatize(word): #приводим слова в нормальную форму
    parse = morph_analyzer.parse(word)[0]
    return(parse.normal_form+'_'+parse.tag.POS)
        
def tokenize(text): #разбиваем текст на слова
    if type(text) == str:
        words = [i for i in word_tokenize(text) if i not in stop_words and not i.isdigit()]
        words = [lemmatize(i) for i in words if morph_analyzer.parse(i)[0].tag.POS != None]
        return words
    else: return ''

In [5]:
df = pd.read_csv('ready_dataset.csv')
df.drop(columns='Unnamed: 0', inplace=True)

In [6]:
df_words = df.copy() #создаем рабочий экзмепляр данных

df_words['words'] = df.msge.apply(tokenize) #токенизируем текст, замещаем в датафрейме

In [22]:
vectorizer = TfidfVectorizer() #tf-idf
dim_reduct = TruncatedSVD(n_components=2) #учим снижать размерность
X = vectorizer.fit_transform(df.msge)
dim_reduct.fit(X)


coords = dim_reduct.transform(X) #снижаем размерность
df_words['tf_idf_x'], df_words['tf_idf_y'] = coords[:, 0], coords[:, 1]

df_words['tf_idf_x'] = df_words['tf_idf_x']+abs(df_words['tf_idf_x'].min()) #нормализуем векторы
df_words['tf_idf_y'] = df_words['tf_idf_y']+abs(df_words['tf_idf_y'].min())
df_words['tf_idf_x'] *= 1/df_words['tf_idf_x'].max()
df_words['tf_idf_y'] *= 1/df_words['tf_idf_y'].max()

In [ ]:
df_words = df_words[df_words.words.apply(len) != 0] #убираем пустые текста

In [ ]:
def get_clean_words_arr(list_of_words):
    clean_list = []
    for i in list_of_words:
        clean_list.append(re.sub(r'(_[^_]*$)', '', i))
    return clean_list

In [ ]:
df_words['words_clean'] = df_words.words.apply(get_clean_words_arr)

In [ ]:
#загрузка w2v модели
model = Word2Vec.load("word2vec.model")

In [ ]:
#модель w2v на данных. Не запускать, если уже имеется
path = get_tmpfile("word2vec.model")
model = Word2Vec(df_words['words_clean'], size=500, window=5, min_count=3, workers=4, sg=0, negative=20)
model.save("word2vec.model")

In [ ]:
all_words_with_meta = [] #создаем промежуточный список для последующего создания датафрейма
for sample in df_words.iterrows(): #идем по датафрейму, выбираем слова, забираем их параметры
    for word in sample[1].words:
        all_words_with_meta.append([word, sample[1]['date'], sample[1]['time'], sample[1]['user']])

In [ ]:
all_words_with_meta = np.array(all_words_with_meta) #переводим в массив numpy
df_words_sep = pd.DataFrame({'word':all_words_with_meta[:, 0],
                        'date':all_words_with_meta[:, 1],
                       'time':all_words_with_meta[:, 2],
                        'user':all_words_with_meta[:, 3]})

In [ ]:
df_words_sep['word_clean'] = df_words_sep.word.apply(lambda x: re.sub(r'(_[^_]*$)', '', x)) #чистим слова от частей речи

In [ ]:
df_words_sep['word_POS_TAG'] = df_words_sep.word.apply(lambda x: re.sub(r'(^[^_]*_)', '', x)) #забираем части речи

In [ ]:
def get_word_embedding(word):
    if word in model.wv.vocab: return model.wv[word]
    else: return np.nan
def get_word_embedding_ruscorpora(word):
    if word in model_ruscorpora.vocab: return model_ruscorpora[word]
    else: return np.nan

In [ ]:
df_words_sep['word_embedding_ruscorpora'] = df_words_sep.word.apply(get_word_embedding_ruscorpora)
df_words_sep['word_embedding'] = df_words_sep.word_clean.apply(get_word_embedding)

In [24]:
df_words_sep.to_pickle('ready_dataset_with_word_embeddings.pkl')

NameError: name 'df_words_sep' is not defined